In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


# 브라우저 창 크기 설정
WINDOW_WIDTH = 1100
WINDOW_HEIGHT = 1000

# Ace Editor 값 가져오기 함수
def get_ace_editor_value(driver, ace_element, new_value):
    try:
            # Ace Editor가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(
                lambda d: d.execute_script("return typeof ace !== 'undefined' && ace.edit(arguments[0]) !== null", ace_element)
            )
            
            # Ace Editor가 화면에 보이도록 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", ace_element)

            # Ace Editor의 기존 값을 복사
            current_value = driver.execute_script("return ace.edit(arguments[0]).getValue();", ace_element).strip()
            print(f"📋 기존 Ace Editor 값 복사: {current_value}")
            extract_value(current_value)
            return current_value  # 복사한 값을 리턴



    except Exception as e:
            print(f"⚠️ Ace Editor 값 설정 실패: {e}")
def extract_value(raw_data):
    # 한글, "컬럼 명", "데이터 타입" 등의 불필요한 텍스트 제거
    cleaned_text = re.sub(r'컬럼 명.*?값', '', raw_data)  # "컬럼 명...값" 패턴 제거
    cleaned_text = re.sub(r'[\xa0\u0590-\u05FF]+', '', cleaned_text)  # 공백 문자 및 히브리어 제거
    cleaned_text = re.sub(r'[X]+$', '', cleaned_text)  # X 문자 제거
    cleaned_text = cleaned_text.strip()  # 좌우 공백 제거
    
    return cleaned_text

# 데이터 추출 함수
def e2(driver, excel_filename):
    data = []
    try:
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div._columnBorder_1rntr_40"))
        )
        time.sleep(1)  # 요소 로드 대기
        for element in elements:
            try:
                column_name = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(1) p").text
                data_type = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(2) p").text
                value = "Placeholder"

                # 데이터 타입에 따라 값 추출 방식 결정
                if data_type == "object":
                    value = get_ace_editor_value(driver,)
                else:
                    try:
                        value = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(3) p").text.strip()
                    except NoSuchElementException:
                        pass

                data.append([column_name, data_type, value])
                print([column_name, data_type, value])

            except NoSuchElementException as e:
                print(f"⚠️ 요소 찾기 실패: {e}")
                continue
    except Exception as e:
        print(f"⚠️ 컬럼 요소를 찾지 못했습니다: {e}")
    return data

# 브라우저 초기화 및 로그인
def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)
    driver.get('https://console.thebackend.io/ko/login')
    time.sleep(0.2)
    return driver

# 로그인 함수
def login_to_console(driver):
    id_input = driver.find_element(By.ID, "username")
    pyperclip.copy("dksrufp0607@naver.com")
    id_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(0.2)

    pw_input = driver.find_element(By.ID, "password")
    pyperclip.copy("ghkfkd159!")
    pw_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(0.2)

    login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
    login_button.click()
    time.sleep(1)
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]'))
    )

# 콘솔 페이지 이동
def navigate_to_page(driver):
    time.sleep(1)
    article_url = 'https://console.thebackend.io/ko/project/1ea3f14d34e89530ea88b3245bc82dc17d5f52ce1554049f19fce9219a847cfce18bb8891c9ffe90bc65e2b9a3b981853fc5513c1dd200afc9590ba6bfd5fced4230647d25328849e0917641/baseGameInfo/data'
    driver.get(article_url)

# "확인" 버튼 클릭
def confirm_click(driver):
    time.sleep(1)
    confirm_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "확인")]'))
    )
    confirm_button.click()
    print("✅ 확인 버튼 클릭 완료!")

# CSV 데이터 로드 함수
def get_csv_value(row, col):
    csv_path = "C:/엑셀/Ranking_1.csv"
    try:
        df = pd.read_csv(csv_path)
        if row < df.shape[0] and col < df.shape[1]:
            print(f"🔹 {row}행 3열 값 (출력용): {df.iloc[row, 2]}")
            return df.iloc[row, col]
        else:
            print("⚠️ CSV 파일의 크기를 초과하는 인덱스입니다.")
            return None
    except Exception as e:
        print(f"⚠️ CSV 로드 중 오류 발생: {e}")
        return None

# 검색 실행 함수
def execute_tasks(driver, row, col):
    b2_value = get_csv_value(row, col)
    if not b2_value:
        return
    time.sleep(1)
    table_dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "_manageInfo-data_1rntr_71")]'))
    )
    table_dropdown.click()
    print("✅ 테이블 드롭다운 클릭 완료!")

    time.sleep(1)
    base_data_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@role="option"][span[text()="BASE_DATA"]]'))
    )
    base_data_option.click()
    print("✅ BASE_DATA 선택 완료!")

    time.sleep(1)
    detail_search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "상세 검색")]'))
    )
    detail_search_button.click()
    print("✅ 상세 검색 클릭 완료!")

    time.sleep(1)
    search_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, "defaultSearchValue"))
    )
    search_input.clear()
    search_input.send_keys(b2_value)
    print(f"✅ B2 값 입력 완료: {b2_value}")

    time.sleep(1)
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui medium positive button") and @type="submit"]'))
    )
    search_button.click()
    print("✅ 검색 버튼 클릭 완료!")

    time.sleep(1)
    uuid_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//p[contains(@class, "_link_a1mor_113")]'))
    )
    uuid_element.click()
    print("✅ UUID 클릭 완료!")

# 엑셀 저장 함수
def save_to_excel(data, filename):
    df = pd.DataFrame(data, columns=["Column Name", "Data Type", "Value"])
    df.to_excel(filename, index=False, engine="openpyxl")
    print(f"✅ 데이터가 {filename}에 저장되었습니다.")

# 메인 실행
if __name__ == "__main__":
    excel_filename = "3300.xlsx"
    data = []

    driver1 = setup_browser(0)
    try:
        login_to_console(driver1)
        navigate_to_page(driver1)
        confirm_click(driver1)
        execute_tasks(driver1, 0, 5)
        data = e2(driver1,excel_filename)
        save_to_excel(data, excel_filename)
    except Exception as e:
        print(f"⚠️ 전체 실행 중 오류: {e}")
    finally:
        time.sleep(1)
       # driver1.quit()
        print("✅ 브라우저 종료")

✅ 확인 버튼 클릭 완료!
🔹 0행 3열 값 (출력용): 슈바니
✅ 테이블 드롭다운 클릭 완료!
✅ BASE_DATA 선택 완료!
✅ 상세 검색 클릭 완료!
✅ B2 값 입력 완료: 0831d8a0-c410-11ef-ac3c-67eaa6cff280
✅ 검색 버튼 클릭 완료!
✅ UUID 클릭 완료!
['event8_Receive', 'number', '39']
📋 기존 Ace Editor 값 복사: 컬럼 명purchaseCurMileage데이터 타입object값12345678910111213[    10,    0,    0,    3,    0,    0,    0,    0,    0,    0,    0]  ההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההההXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

KeyboardInterrupt: 